# Text AST example

This notebook shows an example of what the text AST looks like for a fairly realistic query and demonstrates how the other packages should translate it from/to a Python AST.

## Original query specification

First, let's specify the query. Without importing `func_adl`, I have to put this in as a big string here. In reality, the user would execute the content of this string as a normal Python statement.

In [1]:
query_python_source = """
data_source.Where('lambda e: e.jet_pT.Where(lambda pT: pT > 1000).Count() > 0')\
           .Select('lambda e: [e.eventNumber, e.CalibJet_pT]')
"""

## Workaround to prepare the Python AST

This section is only necessary in this context because I'm not actually using `func_adl`. Some frontend package will do all of this behind the scenes.

The first thing to do is turn this query into a Python AST. The `ast` module can do this via `parse()`.

In [2]:
import ast

In [3]:
query_raw_python_ast = ast.parse(query_python_source)

Now we have a Python AST, but it doesn't know anything about the LINQ-like query nodes (e.g., `Where` and `Select`). The AST needs to be transformed to include these custom nodes. There's a function in `qastle` to accomplish this.

The following is just needed to import the `qastle` module in this notebook:

In [4]:
import os, sys
if os.pardir not in sys.path:
    sys.path.append(os.pardir)

Now import the actual module:

In [5]:
import qastle

Transform the Python AST by inserting LINQ nodes:

In [6]:
query_python_ast = qastle.insert_linq_nodes(query_raw_python_ast)

## Translate from Python AST to text AST

The actual translation step is now as simple as running one function:

In [7]:
text_ast = qastle.python_ast_to_text_ast(query_python_ast)
print(text_ast)

(Select (Where data_source (lambda (list e) (> (Count (Where (attr e 'jet_pT') (lambda (list pT) (> pT 1000)))) 0))) (lambda (list e) (list (attr e 'eventNumber') (attr e 'CalibJet_pT'))))


The above output is the text AST version of the query.

Lastly, the backend will need to be able to take in this text AST string and turn it back into a Python AST that it can use to generate the code that extracts this selection from data. There is another function to handle this:

In [8]:
roundtrip_python_ast = qastle.text_ast_to_python_ast(text_ast)

Finally, we can take a look at the resulting Python AST:

In [9]:
print(ast.dump(roundtrip_python_ast))

Module(body=[Expr(value=Select(source=Where(source=Name(id='data_source', ctx=Load()), predicate=Lambda(args=arguments(args=[arg(arg='e', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=Compare(left=Count(source=Where(source=Attribute(value=Name(id='e', ctx=Load()), attr='jet_pT', ctx=Load()), predicate=Lambda(args=arguments(args=[arg(arg='pT', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=Compare(left=Name(id='pT', ctx=Load()), ops=[Gt()], comparators=[Num(n=1000)])))), ops=[Gt()], comparators=[Num(n=0)]))), selector=Lambda(args=arguments(args=[arg(arg='e', annotation=None)], vararg=None, kwonlyargs=[], kw_defaults=[], kwarg=None, defaults=[]), body=List(elts=[Attribute(value=Name(id='e', ctx=Load()), attr='eventNumber', ctx=Load()), Attribute(value=Name(id='e', ctx=Load()), attr='CalibJet_pT', ctx=Load())], ctx=Load()))))])


...but it's quite difficult to read. As an easy check, we can just verify that it's identical to the original LINQ-query Python AST used to generate the text AST:

In [10]:
ast.dump(roundtrip_python_ast) == ast.dump(query_python_ast)

True